# Teleoperation

In this example we'll control the Jetbot remotely with a gamepad controller connected to our web browser machine.

### Create gamepad controller

The first thing we want to do is create an instance of the ``Controller`` widget, which we'll use to drive our robot.
The ``Controller`` widget takes a ``index`` parameter, which specifies the number of the controller.  This is useful in case you
have multiple controllers attached, or some gamepads *appear* as multiple controllers.  To determine the index
of the controller you're using,

1. Visit [http://html5gamepad.com](http://html5gamepad.com).  
2. Press buttons on the gamepad you're using
3. Remember the ``index`` of the gamepad that is responding to the button presses

Next, we'll create and display our controller using that index.

In [14]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # replace| with index of your controller

display(controller)

Controller()

Even if the index is correct, you may see the text ``Connect gamepad and press any button``.  That's because the gamepad hasn't
registered with this notebook yet.  Press a button and you should see the gamepad widget appear above.

### Connect gamepad controller to robot motors

Now, even though we've connected our gamepad, we haven't yet attached the controls to our robot!  The first, and most simple control
we want to attach is the motor control.  We'll connect that to the left and right vertical axes using the ``dlink`` function.  The
``dlink`` function, unlike the ``link`` function, allows us to attach a transform between the ``source`` and ``target``.  Because
the controller axes are flipped from what we think is intuitive for the motor control, we'll use a small *lambda* function to
negate the value.

> WARNING: This next cell will move the robot if you touch the gamepad controller axes!

In [15]:
from jetracer.nvidia_racecar import NvidiaRacecar
import traitlets

car = NvidiaRacecar()


In [16]:
car.steering_offset=0.0
car.steering = 0.5
car.throttle_gain = 0.4

In [17]:
left_link = traitlets.dlink((controller.axes[0], 'value'), (car, 'steering'), transform=lambda x: x)
right_link = traitlets.dlink((controller.axes[1], 'value'), (car, 'throttle'), transform=lambda x: x)

In [18]:
print(car.__dict__)
traitlets.dlink((controller.buttons[3], 'value'), (car, 'throttle'), transform=lambda x: x)

{'_trait_values': {'i2c_address1': 64, 'i2c_address2': 96, 'steering_channel': 0, 'steering_offset': 0.0, 'steering': 0.0, 'steering_gain': -0.65, 'throttle_gain': 0.4, 'throttle': 0.0}, '_trait_notifiers': {'steering': {'change': [<traitlets.traitlets.ObserveHandler object at 0x7f9c700ac8>]}, 'throttle': {'change': [<traitlets.traitlets.ObserveHandler object at 0x7f9c6b11d0>]}}, '_trait_validators': {'steering': <traitlets.traitlets.ValidateHandler object at 0x7f9c700828>, 'throttle': <traitlets.traitlets.ValidateHandler object at 0x7f9c7006a0>}, '_cross_validation_lock': False, 'kit': <adafruit_servokit.ServoKit object at 0x7f9c680780>, 'motor': <adafruit_servokit.ServoKit object at 0x7f9c55d2e8>, 'steering_motor': <adafruit_motor.servo.ContinuousServo object at 0x7f9c55d7b8>}


Awesome! Our robot should now respond to our gamepad controller movements.  Now we want to view the live video feed from the camera!